In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template(
    "{country_a}와 {country_b}사이의 거리는 얼마나 되니?"
)

prompt = template.format(country_a="멕시코", country_b="한국")

chat.predict(prompt)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 지리학자이다. 오직 {language}로만 대화를 한다"),
    ("ai", "안녕, {name}"), 
    ("human", "{country_a}와 {country_b}사이의 거리는 얼마나 되니? 그리고 당신의 이름은 무엇이니?")  
])

prompt = template.format_messages(
    language="korean", 
    name="조현일", 
    country_a="한국", 
    country_b="일본"
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser 

class CommaOutputParser(BaseOutputParser): 

    def parse(self, text): 
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("Hello, how,are,you")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list생성기계입니다. 입력받은 질문들은 모두 콤마로 구분된 list로 답해야 한다 최대 {max_items}로 답해야 한다."), 
    ("human", "{question}")
])


In [ ]:
chain_one = template | chat | CommaOutputParser() 


chain.invoke({
    "max_items": 5, 
    "question" : "세계적인 자동차는 무엇인가?"
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"), 
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, 
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"), 
    example_prompt, 
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat 

chain.invoke({
    "country": "Korea", 
})

In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.prompts.few_shot import FewShotPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler 

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_template = """
    Human: {question}, 
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {country}?"), 
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt, 
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "한국"})

AIMessageChunk(content='\nI\'m sorry, I don\'t have information on "한국." Can you provide more context or specify which country or region you are referring to?')

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
  examples=examples, 
  example_prompt=example_prompt, 
  max_length=0
)

prompt = FewShotChatMessagePromptTemplate(
  example_prompt = example_prompt, 
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?", 
  input_variables=["country"]

)

prompt.format(country="Brazil")

ValidationError: 3 validation errors for FewShotChatMessagePromptTemplate
example_prompt
  Can't instantiate abstract class BaseMessagePromptTemplate with abstract methods format_messages, input_variables (type=type_error)
example_prompt
  Can't instantiate abstract class BaseChatPromptTemplate with abstract method format_messages (type=type_error)
suffix
  extra fields not permitted (type=value_error.extra)

In [29]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector 



chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector): 

  def __init__(self, examples): 
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables): 
    from random import choice 
    return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
  examples=examples, 
)

prompt = FewShotPromptTemplate(
  example_prompt = example_prompt, 
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?", 
  input_variables=["country"]

)

prompt.format(country="Brazil")

NameError: name 'FewShotPromptTemplate' is not defined

In [22]:
from langchain.chat_models import ChatOpenAI 
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.prompts import load_prompt


prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany") 


'What is the capital of Germany'

In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
  ("intro", intro), 
  ("example", example), 
  ("start", start)
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.globals import set_llm_cache 
from langchain.cache import InMemoryCache, SQLiteCache


set_llm_cache(SQLiteCache("cache.db")) 


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")


In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.callbacks import get_openai_callback 

chat = ChatOpenAI( 
  temperature=0.1
)

with get_openai_callback() as usage: 
  a = chat.predict("What is the recipe for soju")
  b = chat.predict("what is the recipt for bread")
  print(a, b, "\n")
  print(usage)

In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.llms.openai import OpenAI 
from langchain.llms.loading import load_llm

chat = OpenAI(
  temperature=0.1, 
  max_tokens=450, 
  model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

In [ ]:
from langchain.chat_models import ChatOpenAI 
from langchain.llms.openai import OpenAI 
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

# moel/io Challenge 
* 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
* LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
* 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector

# Chat 모델 초기화
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 예시 정의
examples = [
    {
        "question": "'파묘' 영화에 대한 기본 정보를 알려줘?",
        "answer": """
        영화 제목: 파묘 (The Exhumation)
        감독: 김상호
        주요 출연진: 박성웅, 이정은, 김혜수, 류준열
        예산: 약 500만 달러
        흥행 수익: 약 2,000만 달러
        장르: 공포, 미스터리, 스릴러
        시놉시스: 작은 시골 마을에서 오랜 시간 동안 묻혀 있던 묘지가 강제적으로 파헤쳐지며 기이한 사건들이 일어나기 시작합니다. 오래된 전설에 따르면, 이 묘지에는 마을의 어두운 비밀이 숨겨져 있다고 전해지며, 파묘 작업 이후 마을 사람들은 설명할 수 없는 공포와 미스터리에 휩싸이게 됩니다. 주인공은 이 사건의 진실을 밝히기 위해 고군분투하며, 마을의 숨겨진 과거와 얽힌 진실을 파헤쳐 나갑니다.
        """,
    },
    {
        "question": "'변호인' 영화에 대한 기본 정보를 알려줘?",
        "answer": """
        감독: 양우석
        주요 출연진: 송강호, 김영애, 오달수, 곽도원, 임시완
        예산: 약 55억 원
        흥행 수익: 약 1,100억 원 (한국에서 1,130만 명 이상의 관객을 동원)
        장르: 드라마, 법정
        시놉시스: 1980년대 초 부산을 배경으로, 송우석 변호사는 세금 전문 변호사로 성공을 거두고 부를 쌓고 있습니다. 그러나 그의 대학 동창의 아들이 국가보안법 위반 혐의로 체포되고 고문당하는 사건을 접하게 됩니다. 송우석은 이를 계기로 인권 변호사로서의 길을 걷게 되고, 불의에 맞서 싸우며 정의를 수호하려 합니다. 이 영화는 실존 인물인 노무현 전 대통령의 초창기 변호사 시절을 모티브로 하여 제작되었습니다.
        """,
    },
    {
        "question": "'극한직업' 영화에 대한 기본 정보를 알려줘?",
        "answer": """
        주요 출연진: 류승룡, 이하늬, 진선규, 이동휘, 공명
        예산: 약 65억 원
        흥행 수익: 약 1,500억 원 (한국에서 1,626만 명 이상의 관객을 동원하며 역대 흥행 2위 기록)
        장르: 코미디, 범죄
        시놉시스: 마약반 형사들은 범죄 조직을 소탕하기 위해 잠복 근무를 시작하며 치킨집을 위장 창업하게 됩니다. 그런데 이 치킨집이 의외로 큰 인기를 끌게 되면서 형사들은 본업인 수사와 치킨집 운영 사이에서 갈등하게 됩니다. 코미디와 액션이 어우러진 이 영화는 웃음을 주는 동시에 예기치 못한 상황에서의 긴장감을 제공합니다.
        """,
    },
]

# PromptTemplate 생성
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# ExampleSelector 설정
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=200  # 합리적인 길이로 설정
)

# ChatPromptTemplate 설정
prompt = ChatPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {movie} 영화에 대한 기본 정보를 알려줘?",
    input_variables=["movie"]
)

# 프롬프트 포맷
formatted_prompt = prompt.format(movie="서울의 봄")
print(formatted_prompt)

KeyError: 'messages'

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "파묘",
        "answer": """
        영화 제목: 파묘 (The Exhumation)
        감독: 김상호
        주요 출연진: 박성웅, 이정은, 김혜수, 류준열
        예산: 약 500만 달러
        흥행 수익: 약 2,000만 달러
        장르: 공포, 미스터리, 스릴러
        시놉시스: 작은 시골 마을에서 오랜 시간 동안 묻혀 있던 묘지가 강제적으로 파헤쳐지며 기이한 사건들이 일어나기 시작합니다. 오래된 전설에 따르면, 이 묘지에는 마을의 어두운 비밀이 숨겨져 있다고 전해지며, 파묘 작업 이후 마을 사람들은 설명할 수 없는 공포와 미스터리에 휩싸이게 됩니다. 주인공은 이 사건의 진실을 밝히기 위해 고군분투하며, 마을의 숨겨진 과거와 얽힌 진실을 파헤쳐 나갑니다.
        """,
    },
    {
        "movie": "변호인",
        "answer": """
        감독: 양우석
        주요 출연진: 송강호, 김영애, 오달수, 곽도원, 임시완
        예산: 약 55억 원
        흥행 수익: 약 1,100억 원 (한국에서 1,130만 명 이상의 관객을 동원)
        장르: 드라마, 법정
        시놉시스: 1980년대 초 부산을 배경으로, 송우석 변호사는 세금 전문 변호사로 성공을 거두고 부를 쌓고 있습니다. 그러나 그의 대학 동창의 아들이 국가보안법 위반 혐의로 체포되고 고문당하는 사건을 접하게 됩니다. 송우석은 이를 계기로 인권 변호사로서의 길을 걷게 되고, 불의에 맞서 싸우며 정의를 수호하려 합니다. 이 영화는 실존 인물인 노무현 전 대통령의 초창기 변호사 시절을 모티브로 하여 제작되었습니다.
        """,
    },
    {
        "movie": "극한직업",
        "answer": """
        주요 출연진: 류승룡, 이하늬, 진선규, 이동휘, 공명
        예산: 약 65억 원
        흥행 수익: 약 1,500억 원 (한국에서 1,626만 명 이상의 관객을 동원하며 역대 흥행 2위 기록)
        장르: 코미디, 범죄
        시놉시스: 마약반 형사들은 범죄 조직을 소탕하기 위해 잠복 근무를 시작하며 치킨집을 위장 창업하게 됩니다. 그런데 이 치킨집이 의외로 큰 인기를 끌게 되면서 형사들은 본업인 수사와 치킨집 운영 사이에서 갈등하게 됩니다. 코미디와 액션이 어우러진 이 영화는 웃음을 주는 동시에 예기치 못한 상황에서의 긴장감을 제공합니다.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "{movie} 영화에 대한 기본 정보를 알려줘?"), 
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt, 
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "what do you know about {movie}"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "marvel"})

Marvel is a popular American entertainment company known for its comic books, movies, and TV shows featuring superheroes. Some of the most famous Marvel characters include Spider-Man, Iron Man, Captain America, Thor, Black Widow, and the Hulk. Marvel Studios produces the Marvel Cinematic Universe (MCU), a series of interconnected superhero films that have been highly successful at the box office.

AIMessageChunk(content='Marvel is a popular American entertainment company known for its comic books, movies, and TV shows featuring superheroes. Some of the most famous Marvel characters include Spider-Man, Iron Man, Captain America, Thor, Black Widow, and the Hulk. Marvel Studios produces the Marvel Cinematic Universe (MCU), a series of interconnected superhero films that have been highly successful at the box office.')